# ACA Premium Tax Credit Analysis - California Couple
## 700% FPL Extension with Amended Rate Schedule

Analysis for blog post: [Analyzing the Bipartisan Health Insurance Affordability Act](https://policyengine.org/us/research/bipartisan-health-insurance-affordability-act)

In [12]:
from policyengine_us import Simulation
from policyengine_core.reforms import Reform
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from policyengine_core.charts import format_fig

## Define Reform

### 700% FPL Extension with Amended Rate Schedule

| FPL Range | Initial % | Final % |
|-----------|-----------|--------|
| 200-250%  | 2.0%      | 4.0%   |
| 250-300%  | 4.0%      | 6.0%   |
| 300-400%  | 6.0%      | 8.5%   |
| 400-600%  | 8.5%      | 8.5%   |
| 600-700%  | 8.5%      | 9.25%  |

Eligibility cuts off at 700% FPL.

In [13]:
# Import the 700% FPL cliff reform
from policyengine_us.reforms.aca.aca_ptc_700_fpl_cliff import aca_ptc_700_fpl_cliff

reform_700fpl = aca_ptc_700_fpl_cliff

# IRA Extension reform (extends current 2021-2025 subsidies indefinitely)
reform_ira = Reform.from_dict({
    "gov.aca.required_contribution_percentage[0].amount": {
        "2026-01-01.2100-12-31": 0
    },
    "gov.aca.required_contribution_percentage[1].amount": {
        "2026-01-01.2100-12-31": 0
    },
    "gov.aca.required_contribution_percentage[2].amount": {
        "2026-01-01.2100-12-31": 0
    },
    "gov.aca.required_contribution_percentage[3].amount": {
        "2026-01-01.2100-12-31": 0.02
    },
    "gov.aca.required_contribution_percentage[4].amount": {
        "2026-01-01.2100-12-31": 0.04
    },
    "gov.aca.required_contribution_percentage[5].amount": {
        "2026-01-01.2100-12-31": 0.06
    },
    "gov.aca.required_contribution_percentage[6].amount": {
        "2026-01-01.2100-12-31": 0.085
    },
    "gov.aca.ptc_income_eligibility[2].amount": {
        "2026-01-01.2100-12-31": True
    }
}, country_id="us")

## Define California Household

Couple in San Benito County, CA:
- Person 1: age 64
- Person 2: age 62

In [14]:
situation_california = {
    "people": {
        "you": {
            "age": {"2026": 64}
        },
        "your partner": {
            "age": {"2026": 62}
        }
    },
    "families": {
        "your family": {
            "members": ["you", "your partner"]
        }
    },
    "spm_units": {
        "your household": {
            "members": ["you", "your partner"]
        }
    },
    "tax_units": {
        "your tax unit": {
            "members": ["you", "your partner"]
        }
    },
    "households": {
        "your household": {
            "members": ["you", "your partner"],
            "state_name": {"2026": "CA"},
            "county_fips": {"2026": "06069"}  # San Benito County
        }
    },
    "marital_units": {
        "your marital unit": {
            "members": ["you", "your partner"]
        }
    },
    "axes": [[
        {
            "name": "employment_income",
            "count": 800,
            "min": 0,
            "max": 200000
        }
    ]]
}

## Create Simulations

In [15]:
# Baseline simulation
simulation_baseline = Simulation(situation=situation_california)

# 700% FPL Extension reform
simulation_700fpl = Simulation(situation=situation_california, reform=reform_700fpl)

# IRA Extension reform
simulation_ira = Simulation(situation=situation_california, reform=reform_ira)

## Calculate Benefits and Net Income

In [16]:
# Get household-level values
household_income = simulation_baseline.calculate("employment_income", map_to="household", period=2026)

# Baseline
baseline_aca_ptc = simulation_baseline.calculate("aca_ptc", map_to="household", period=2026)
baseline_net_income = simulation_baseline.calculate(
    "household_net_income_including_health_benefits", 
    map_to="household", 
    period=2026
)

# 700% FPL Extension
reform_700fpl_aca_ptc = simulation_700fpl.calculate("aca_ptc", map_to="household", period=2026)
reform_700fpl_net_income = simulation_700fpl.calculate(
    "household_net_income_including_health_benefits", 
    map_to="household", 
    period=2026
)

# IRA Extension
reform_ira_aca_ptc = simulation_ira.calculate("aca_ptc", map_to="household", period=2026)
reform_ira_net_income = simulation_ira.calculate(
    "household_net_income_including_health_benefits", 
    map_to="household", 
    period=2026
)

## Chart: Health Programs by Income Level

Shows all health assistance programs (Medicaid, CHIP, ACA PTC) across income levels for baseline and both reform scenarios.

In [14]:
# Calculate additional health program benefits for the programs chart
baseline_medicaid = simulation_baseline.calculate("medicaid_cost", map_to="household", period=2026)
baseline_chip = simulation_baseline.calculate("per_capita_chip", map_to="household", period=2026)

reform_700fpl_medicaid = simulation_700fpl.calculate("medicaid_cost", map_to="household", period=2026)
reform_700fpl_chip = simulation_700fpl.calculate("per_capita_chip", map_to="household", period=2026)

reform_ira_medicaid = simulation_ira.calculate("medicaid_cost", map_to="household", period=2026)
reform_ira_chip = simulation_ira.calculate("per_capita_chip", map_to="household", period=2026)

# Calculate totals
baseline_total = baseline_aca_ptc + baseline_medicaid + baseline_chip
reform_700fpl_total = reform_700fpl_aca_ptc + reform_700fpl_medicaid + reform_700fpl_chip
reform_ira_total = reform_ira_aca_ptc + reform_ira_medicaid + reform_ira_chip

# Colors
GRAY = "#808080"
BLUE_PRIMARY = "#2C6496"
PURPLE = "#9467BD"
TEAL_ACCENT = "#39C6C0"
DARK_GRAY = "#616161"

# Create health programs chart
fig_programs = go.Figure()

# Medicaid doesn't change with PTC reforms - show only once
fig_programs.add_trace(go.Scatter(
    x=household_income,
    y=baseline_medicaid,
    mode='lines',
    name='Medicaid',
    line=dict(color=TEAL_ACCENT, width=2)
))

# ACA PTC varies by scenario
fig_programs.add_trace(go.Scatter(
    x=household_income,
    y=baseline_aca_ptc,
    mode='lines',
    name='ACA PTC (Baseline)',
    line=dict(color=DARK_GRAY, width=2)
))

fig_programs.add_trace(go.Scatter(
    x=household_income,
    y=reform_700fpl_aca_ptc,
    mode='lines',
    name='ACA PTC (700% FPL)',
    line=dict(color=BLUE_PRIMARY, width=2)
))

fig_programs.add_trace(go.Scatter(
    x=household_income,
    y=reform_ira_aca_ptc,
    mode='lines',
    name='ACA PTC (IRA)',
    line=dict(color=PURPLE, width=2)
))

# Total benefit lines
fig_programs.add_trace(go.Scatter(
    x=household_income,
    y=baseline_total,
    mode='lines',
    name='Total Benefits (Baseline)',
    line=dict(color=DARK_GRAY, width=2, dash='longdash'),
    visible='legendonly'
))

fig_programs.add_trace(go.Scatter(
    x=household_income,
    y=reform_700fpl_total,
    mode='lines',
    name='Total Benefits (700% FPL)',
    line=dict(color=BLUE_PRIMARY, width=2, dash='longdash'),
    visible='legendonly'
))

fig_programs.add_trace(go.Scatter(
    x=household_income,
    y=reform_ira_total,
    mode='lines',
    name='Total Benefits (IRA)',
    line=dict(color=PURPLE, width=2, dash='longdash'),
    visible='legendonly'
))

fig_programs.update_layout(
    title='California Couple (ages 64 & 62) - Health Programs by Income Level',
    xaxis_title='Household Income',
    yaxis_title='Annual Benefit Amount',
    xaxis=dict(tickformat='$,.0f', range=[0, 200000]),
    yaxis=dict(tickformat='$,.0f'),
    height=600,
    width=1000,
    legend=dict(orientation='h', yanchor='top', y=-0.15, xanchor='center', x=0.5)
)

fig_programs = format_fig(fig_programs)
fig_programs.show()

## Visualization Setup

In [17]:
# Color scheme
GRAY = "#808080"
BLUE_PRIMARY = "#2C6496"
PURPLE = "#9467BD"

## Chart 1: ACA Premium Tax Credit by Income Level

In [18]:
fig_ptc = go.Figure()

# Baseline
fig_ptc.add_trace(go.Scatter(
    x=household_income,
    y=baseline_aca_ptc,
    mode='lines',
    name='Baseline',
    line=dict(color=GRAY, width=2)
))

# 700% FPL Extension
fig_ptc.add_trace(go.Scatter(
    x=household_income,
    y=reform_700fpl_aca_ptc,
    mode='lines',
    name='700% FPL Extension',
    line=dict(color=BLUE_PRIMARY, width=2)
))

# IRA Extension
fig_ptc.add_trace(go.Scatter(
    x=household_income,
    y=reform_ira_aca_ptc,
    mode='lines',
    name='IRA Extension',
    line=dict(color=PURPLE, width=2)
))

fig_ptc.update_layout(
    title='California Couple (ages 64 & 62) - ACA Premium Tax Credit by Income',
    xaxis_title='Household Income',
    yaxis_title='ACA Premium Tax Credit',
    xaxis=dict(tickformat='$,.0f', range=[0, 200000]),
    yaxis=dict(tickformat='$,.0f'),
    height=600,
    width=1000,
    legend=dict(orientation='h')
)

fig_ptc = format_fig(fig_ptc)
fig_ptc.show()

## Chart 2: Health-Adjusted Net Income

In [19]:
fig_net_income = go.Figure()

# Baseline
fig_net_income.add_trace(go.Scatter(
    x=household_income,
    y=baseline_net_income,
    mode='lines',
    name='Baseline',
    line=dict(color=GRAY, width=2)
))

# 700% FPL Extension
fig_net_income.add_trace(go.Scatter(
    x=household_income,
    y=reform_700fpl_net_income,
    mode='lines',
    name='700% FPL Extension',
    line=dict(color=BLUE_PRIMARY, width=2)
))

# IRA Extension
fig_net_income.add_trace(go.Scatter(
    x=household_income,
    y=reform_ira_net_income,
    mode='lines',
    name='IRA Extension',
    line=dict(color=PURPLE, width=2)
))

fig_net_income.update_layout(
    title='California Couple (ages 64 & 62) - Health-Adjusted Net Income',
    xaxis_title='Household Income',
    yaxis_title='Health-Adjusted Net Income',
    xaxis=dict(tickformat='$,.0f', range=[0, 200000]),
    yaxis=dict(tickformat='$,.0f'),
    height=600,
    width=1000,
    legend=dict(orientation='h')
)

fig_net_income = format_fig(fig_net_income)
fig_net_income.show()

## Chart 3: Impact on Net Income (Reform - Baseline)

In [20]:
# Calculate deltas
delta_700fpl = reform_700fpl_net_income - baseline_net_income
delta_ira = reform_ira_net_income - baseline_net_income

fig_delta = go.Figure()

# 700% FPL Extension impact
fig_delta.add_trace(go.Scatter(
    x=household_income,
    y=delta_700fpl,
    mode='lines',
    name='700% FPL Extension',
    line=dict(color=BLUE_PRIMARY, width=2)
))

# IRA Extension impact
fig_delta.add_trace(go.Scatter(
    x=household_income,
    y=delta_ira,
    mode='lines',
    name='IRA Extension',
    line=dict(color=PURPLE, width=2)
))

fig_delta.update_layout(
    title='California Couple (ages 64 & 62) - Impact of Premium Tax Credit Reforms',
    xaxis_title='Household Income',
    yaxis_title='Change in Net Income',
    xaxis=dict(tickformat='$,.0f', range=[0, 200000]),
    yaxis=dict(
        tickformat='$,.0f',
        zeroline=True,
        zerolinewidth=1,
        zerolinecolor='black'
    ),
    height=600,
    width=1000,
    legend=dict(orientation='h')
)

fig_delta = format_fig(fig_delta)
fig_delta.show()

## Chart 4: Marginal Tax Rates

In [21]:
# Create situation for MTR calculation with more points
situation_ca_for_mtr = {
    "people": {
        "you": {
            "age": {"2026": 64},
            "employment_income": {"2026": 0},
            "self_employment_income": {"2026": 0}
        },
        "your partner": {
            "age": {"2026": 62},
            "employment_income": {"2026": 0},
            "self_employment_income": {"2026": 0}
        }
    },
    "families": {
        "your family": {
            "members": ["you", "your partner"]
        }
    },
    "spm_units": {
        "your household": {
            "members": ["you", "your partner"]
        }
    },
    "tax_units": {
        "your tax unit": {
            "members": ["you", "your partner"]
        }
    },
    "households": {
        "your household": {
            "members": ["you", "your partner"],
            "state_name": {"2026": "CA"},
            "county_fips": {"2026": "06069"}
        }
    },
    "marital_units": {
        "your marital unit": {
            "members": ["you", "your partner"]
        }
    },
    "axes": [[
        {
            "name": "employment_income",
            "min": 0,
            "max": 200000,
            "count": 400,
            "period": "2026"
        }
    ]]
}

# Calculate net incomes for MTR
sim_baseline_mtr = Simulation(situation=situation_ca_for_mtr)
sim_700fpl_mtr = Simulation(situation=situation_ca_for_mtr, reform=reform_700fpl)
sim_ira_mtr = Simulation(situation=situation_ca_for_mtr, reform=reform_ira)

household_income_mtr = sim_baseline_mtr.calculate("employment_income", map_to="household", period=2026)
baseline_net_income_mtr = sim_baseline_mtr.calculate(
    "household_net_income_including_health_benefits", 
    map_to="household", 
    period=2026
)
reform_700fpl_net_income_mtr = sim_700fpl_mtr.calculate(
    "household_net_income_including_health_benefits", 
    map_to="household", 
    period=2026
)
reform_ira_net_income_mtr = sim_ira_mtr.calculate(
    "household_net_income_including_health_benefits", 
    map_to="household", 
    period=2026
)

# Function to calculate MTR
def calc_mtr(incomes, net_incomes):
    """Calculate MTR between adjacent income points."""
    mtrs = []
    mtr_incomes = []
    for i in range(len(incomes) - 1):
        income_change = incomes[i + 1] - incomes[i]
        net_change = net_incomes[i + 1] - net_incomes[i]
        if income_change > 0 and not np.isnan(net_incomes[i]) and not np.isnan(net_incomes[i + 1]):
            mtr = 1 - (net_change / income_change)
            mtrs.append(mtr)
            mtr_incomes.append((incomes[i] + incomes[i + 1]) / 2)
    return np.array(mtr_incomes), np.array(mtrs)

baseline_mtr_incomes, baseline_mtrs = calc_mtr(household_income_mtr, baseline_net_income_mtr)
reform_700fpl_mtr_incomes, reform_700fpl_mtrs = calc_mtr(household_income_mtr, reform_700fpl_net_income_mtr)
reform_ira_mtr_incomes, reform_ira_mtrs = calc_mtr(household_income_mtr, reform_ira_net_income_mtr)

# Create MTR chart
fig_mtr = go.Figure()

fig_mtr.add_trace(go.Scatter(
    x=baseline_mtr_incomes,
    y=np.clip(baseline_mtrs, -1.0, 1.0),
    mode='lines',
    name='Baseline',
    line=dict(color=GRAY, width=2),
    hovertemplate='Income: $%{x:,.0f}<br>Baseline MTR: %{y:.1%}<extra></extra>'
))

fig_mtr.add_trace(go.Scatter(
    x=reform_700fpl_mtr_incomes,
    y=np.clip(reform_700fpl_mtrs, -1.0, 1.0),
    mode='lines',
    name='700% FPL Extension',
    line=dict(color=BLUE_PRIMARY, width=2),
    hovertemplate='Income: $%{x:,.0f}<br>700% FPL MTR: %{y:.1%}<extra></extra>'
))

fig_mtr.add_trace(go.Scatter(
    x=reform_ira_mtr_incomes,
    y=np.clip(reform_ira_mtrs, -1.0, 1.0),
    mode='lines',
    name='IRA Extension',
    line=dict(color=PURPLE, width=2),
    hovertemplate='Income: $%{x:,.0f}<br>IRA MTR: %{y:.1%}<extra></extra>'
))

fig_mtr.update_layout(
    title='Marginal Tax Rate (including health benefits) - California Couple (ages 64 & 62)',
    xaxis_title='Employment Income',
    yaxis_title='Marginal Tax Rate',
    xaxis=dict(
        tickformat='$,.0f',
        range=[0, 200000],
        gridcolor='lightgray',
        showgrid=True
    ),
    yaxis=dict(
        tickformat='.0%',
        range=[-1.0, 1.0],
        gridcolor='lightgray',
        showgrid=True,
        zeroline=True,
        zerolinewidth=1,
        zerolinecolor='gray'
    ),
    height=600,
    width=1000,
    hovermode='x unified',
    plot_bgcolor='white',
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    )
)

fig_mtr = format_fig(fig_mtr)
fig_mtr.show()

## Key Income Benchmarks Analysis

Calculate PTC at specific FPL percentages (138%, 300%, 400%, 600%, 700%)

In [22]:
import copy
import pandas as pd

PERIOD = 2026

def get_tax_unit_fpg(base_situation: dict, period=PERIOD) -> float:
    """Return the tax unit FPG for the given situation/year."""
    sit = copy.deepcopy(base_situation)
    sit.pop("axes", None)
    
    for person in sit["people"].values():
        person.setdefault("employment_income", {})
        person["employment_income"][str(period)] = 0
    
    sim = Simulation(situation=sit)
    fpg = sim.calculate("tax_unit_fpg", map_to="tax_unit", period=period)[0]
    return float(fpg)

def calc_ptc_for_income(base_situation: dict, income: float, *, reform_to_use=None, period=PERIOD):
    """Return ACA PTC for the given income and year."""
    sit = copy.deepcopy(base_situation)
    sit.pop("axes", None)
    
    # Split income between the two adults
    sit["people"]["you"]["employment_income"] = {str(period): income / 2}
    sit["people"]["your partner"]["employment_income"] = {str(period): income / 2}
    
    sim = Simulation(situation=sit, reform=reform_to_use)
    return sim.calculate("aca_ptc", map_to="household", period=period)[0]

# Get FPG for this household
fpg_2026 = get_tax_unit_fpg(situation_california, period=PERIOD)

# Define income levels to test
percent_targets = {
    "138% FPL": 1.38,
    "300% FPL": 3.00,
    "400% FPL": 4.00,
    "600% FPL": 6.00,
    "700% FPL": 7.00,
}

rows = []
for label, mult in percent_targets.items():
    inc = round(fpg_2026 * mult, 2)
    rows.append({
        "income_label": f"{label} (${inc:,.2f})",
        "income_usd": inc,
        "ptc_baseline": calc_ptc_for_income(situation_california, inc, reform_to_use=None, period=PERIOD),
        "ptc_700fpl_extension": calc_ptc_for_income(situation_california, inc, reform_to_use=reform_700fpl, period=PERIOD),
        "ptc_ira_extension": calc_ptc_for_income(situation_california, inc, reform_to_use=reform_ira, period=PERIOD),
    })

ptc_df = pd.DataFrame(rows)
ptc_df

,income_label,income_usd,ptc_baseline,ptc_700fpl_extension,ptc_ira_extension
0,"138% FPL ($29,897.10)",29897.10,0.000000,0.000000,0.000000
1,"300% FPL ($64,993.70)",64993.70,36241.796875,38701.808594,38701.808594
2,"400% FPL ($86,658.27)",86658.27,0.000000,35349.214844,35349.214844
3,"600% FPL ($129,987.40)",129987.40,0.000000,31529.753906,31666.238281
4,"700% FPL ($151,651.96)",151651.96,0.000000,0.000000,29824.751953


## Health Programs at Key Income Levels

Medicaid benefits at the same FPL thresholds (baseline scenario - these don't change with PTC reforms).

In [ ]:
# Calculate Medicaid at key income levels (no CHIP for this adult couple)
def calc_health_programs_for_income(base_situation: dict, income: float, period=PERIOD):
    """Return Medicaid value for the given income."""
    sit = copy.deepcopy(base_situation)
    sit.pop("axes", None)
    
    # Split income between the two adults
    sit["people"]["you"]["employment_income"] = {str(period): income / 2}
    sit["people"]["your partner"]["employment_income"] = {str(period): income / 2}
    
    sim = Simulation(situation=sit)
    medicaid = sim.calculate("medicaid_cost", map_to="household", period=period)[0]
    return medicaid

# Build table with all health programs at key income levels
health_rows = []
for label, mult in percent_targets.items():
    inc = round(fpg_2026 * mult, 2)
    medicaid = calc_health_programs_for_income(situation_california, inc)
    
    # Get PTC values from the earlier calculation
    ptc_baseline = calc_ptc_for_income(situation_california, inc, reform_to_use=None, period=PERIOD)
    ptc_700fpl = calc_ptc_for_income(situation_california, inc, reform_to_use=reform_700fpl, period=PERIOD)
    ptc_ira = calc_ptc_for_income(situation_california, inc, reform_to_use=reform_ira, period=PERIOD)
    
    health_rows.append({
        "Income Level": label,
        "Income ($)": f"${inc:,.0f}",
        "Medicaid": f"${medicaid:,.0f}",
        "PTC (Baseline)": f"${ptc_baseline:,.0f}",
        "PTC (700% FPL)": f"${ptc_700fpl:,.0f}",
        "PTC (IRA)": f"${ptc_ira:,.0f}",
    })

health_df = pd.DataFrame(health_rows)
health_df